The first step is a bit of setup: this part sets constants, runs basic housekeeping commands, imports necessary packages, and requests the input file from the user.

In [ ]:
TRADING_DAYS_PER_YEAR = 365 # @param {"type":"integer","placeholder":"365"}
CONTRACT_SIZE = 1.0 # @param {"type":"number","placeholder":"1.0"}
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from google.colab import files

btfile = files.upload()

This part reads the CSV file and converts it into a sorted dataframe.

In [ ]:
df = pd.read_csv(list(btfile.keys())[0], parse_dates=["date", "expiry"]).sort_values(["date", "instrument_id"])

This sets proper types for the columns.

In [ ]:
floats = ["strike", "price_bid", "price_ask", "position"]
for c in floats:
    if c in df.columns:
      df[c] = df[c].astype(float)
df["type"] = df["type"].astype(str)
if "underlying" not in df.columns:
    raise ValueError("CSV must include an 'underlying' column")

Next, we need to set the index.

In [ ]:
df = df.set_index(["date", "instrument_id"])

If the user attempts to take a position in an option that has already expired, an error is produced.

In [ ]:
m = df["expiry"].notna() & (df.index.get_level_values("date") > df["expiry"])
if (m & (df["position"] != 0) & (df["type"] != "S")).any():
    raise ValueError("Attempted position for expired option")

This part finds the midpoint price for each asset on each day. It then initializes the mark price to be equal to this midpoint price.

In [ ]:
df["mid"] = (df["price_bid"] + df["price_ask"]) / 2
df["mark"] = df["mid"]

Build a lookup table of spot mid prices (for auto-exercise at expiry).

In [ ]:
spot_rows = df["type"] == "S"
spot_mid = (
    df.loc[spot_rows, ["mid"]]
      .reset_index()
      .rename(columns={"instrument_id": "underlying",
                       "mid": "spot_mid"})
)

Merge spot mid prices into corresponding option rows (for auto-exercise at expiry).

In [ ]:
option_rows = df["type"].isin(["C", "P"])
opts = (
    df.loc[option_rows]
      .reset_index()
      .merge(spot_mid, on=["date", "underlying"], how="left")
)

if opts["spot_mid"].isna().any():
    missing = opts[opts["spot_mid"].isna()][["date", "instrument_id", "underlying"]]
    raise ValueError(f"Missing spot price for some option rows:\n{missing}")

Intrinsic is computed from the underlying spot mid.

In [ ]:
intrinsic = np.zeros(len(df))
idx = opts.index

call = opts["type"] == "C"
put = opts["type"] == "P"

intrinsic[idx[call]] = np.maximum(0, opts.loc[call, "spot_mid"] - opts.loc[call, "strike"])
intrinsic[idx[put]] = np.maximum(0, opts.loc[put,  "strike"] - opts.loc[put,  "spot_mid"])

df["intrinsic"] = intrinsic

The mark price is treated as zero on the day of an option's expiration (instead, as indicated below, it is auto-exercised).

In [ ]:
exp_day = df["type"].isin(["C", "P"]) & (df.index.get_level_values("date") == df["expiry"])
df.loc[exp_day, "mark"] = 0.0

These are shifted versions of the position and mark price columns, which are used to determine changes in those values.

In [ ]:
pos_prev = df.groupby(level=1)["position"].shift(1).fillna(0)
mark_prev = df.groupby(level=1)["mark"].shift(1)

This calculates the change in mark price for pre-existing positions.

In [ ]:
price_leg = (df["mark"] - mark_prev).fillna(0) * pos_prev * CONTRACT_SIZE

This auto-exercises options at expiry.

In [ ]:
expiry_hit = df["type"].isin(["C", "P"]) & (df.index.get_level_values("date") == df["expiry"])
exercise_leg = np.where(expiry_hit, df["intrinsic"] * pos_prev * CONTRACT_SIZE, 0.0)

This accounts for bid-ask spread.

In [ ]:
delta = df["position"] - pos_prev
spread_half = (df["price_ask"] - df["price_bid"]) / 2
cost = delta.abs() * spread_half * CONTRACT_SIZE

This calculates the per-day PnL.

In [ ]:
pnl = (price_leg + exercise_leg - cost).groupby(level=0).sum()

Now, we can calculate the annualized Sharpe ratio.

In [ ]:
mu, sigma = pnl.mean(), pnl.std(ddof=0)
sharpe = np.nan if sigma == 0 else mu / sigma * np.sqrt(TRADING_DAYS_PER_YEAR)
print(f"Sharpe ratio: {sharpe:.4f}")

Finally, the cumulative PnL can be displayed as a graph.

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(pnl.cumsum().index, pnl.cumsum().values)
plt.title("Cumulative PnL")
plt.xlabel("Date")
plt.ylabel("PnL")
plt.grid(True, linestyle="--", alpha=0.6)
plt.tight_layout()
plt.show()